## Setup & Installation

In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain-openai langchain-community langchain-core tavily-python pydantic python-docx docx2pdf reportlab

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.1.2 which is incompatible.
langchain 0.3.27 requires langchain-text-splitters<1.0.0,>=0.3.9, but you have langchain-text-splitters 1.0.0 which is incompatible.


In [2]:
import os
import getpass
from typing import List, Dict, Any, Set
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from IPython.display import Markdown, display
import json

## API Keys Configuration

In [3]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")

print("API keys configured!")

API keys configured!


## Data Models

In [4]:
class Activity(BaseModel):
    """Single activity in the itinerary"""
    name: str = Field(description="Activity name")
    description: str = Field(description="Brief description")
    location: str = Field(description="Specific location")
    duration_hours: float = Field(description="Duration in hours")
    estimated_cost: float = Field(description="Estimated cost in USD")
    time_of_day: str = Field(description="Morning/Afternoon/Evening")
    category: str = Field(description="Food/Sightseeing/Activity/Transport")

class DayPlan(BaseModel):
    """Plan for a single day"""
    day_number: int
    activities: List[Activity]
    total_cost: float

class ConflictResolution(BaseModel):
    """Instructions for resolving conflicts in the itinerary"""
    has_conflicts: bool = Field(description="Whether conflicts were found")
    issues_found: List[str] = Field(description="List of issues identified")
    resolutions: List[str] = Field(description="Actions to fix the issues")

print("Data models defined!")

Data models defined!


## Initialize AI Models

In [5]:
# Initialize ChatGPT
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Initialize search tool
tavily_search = TavilySearchResults(max_results=5)

print("AI models initialized!")

AI models initialized!


C:\Users\DELL\AppData\Local\Temp\ipykernel_6312\4135260133.py:5: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tavily_search = TavilySearchResults(max_results=5)


## Helper Functions

In [6]:
def query_destination_guide(destination: str, budget_per_day: float) -> str:
    """Query destination information"""
    query = f"{destination} travel guide budget ${budget_per_day:.0f} per day attractions"
    try:
        results = tavily_search.invoke(query)
        return "\n\n".join([f"Source: {doc['url']}\n{doc['content']}" for doc in results])
    except Exception as e:
        print(f"Search failed: {e}")
        return f"General information about {destination}."

def get_visited_locations(daily_plans: List[DayPlan]) -> Set[str]:
    """Extract all unique locations visited so far"""
    locations = set()
    for day in daily_plans:
        for activity in day.activities:
            # Normalize location names for comparison
            loc = activity.location.lower().strip()
            locations.add(loc)
    return locations

def plan_single_day(destination: str, day_number: int, num_days: int,
                     daily_budget: float, guide: str,
                     visited_locations: Set[str] = None,
                     target_utilization: float = 0.92) -> DayPlan:
    """Plan activities for a single day avoiding previously visited locations"""
    if visited_locations is None:
        visited_locations = set()

    target_cost = daily_budget * target_utilization

    visited_list = "\n".join([f"- {loc}" for loc in visited_locations]) if visited_locations else "None yet"

    planning_prompt = f"""Plan Day {day_number} of a {num_days}-day trip to {destination}.

Daily Budget: ${daily_budget:.2f}
TARGET COST: ${target_cost:.2f} (aim to use 90-95% of budget)
Guide Info: {guide[:1000]}...

LOCATIONS ALREADY VISITED (DO NOT REPEAT):
{visited_list}

CRITICAL RULES:
1. DO NOT visit any location from the "already visited" list
2. Choose DIFFERENT attractions, restaurants, and areas
3. Each location must be UNIQUE - not visited on previous days
4. Total cost should be CLOSE TO ${target_cost:.2f} (90-95% of budget)

Create a realistic itinerary with:
- 4-6 activities (meals at DIFFERENT restaurants, NEW sightseeing spots, transport)
- UNIQUE locations not in the visited list
- Specific location names
- Logical morning to afternoon to evening flow
- Quality activities that maximize budget value
"""

    structured_llm = llm.with_structured_output(DayPlan)
    day_plan = structured_llm.invoke([
        SystemMessage(content=planning_prompt),
        HumanMessage(content=f"Create day {day_number} plan with NEW locations only. Aim for ${target_cost:.2f}.")
    ])

    # Ensure budget utilization
    if day_plan.total_cost < daily_budget * 0.85:
        scale = target_cost / day_plan.total_cost if day_plan.total_cost > 0 else 1
        for activity in day_plan.activities:
            activity.estimated_cost *= scale
        day_plan.total_cost = min(target_cost, daily_budget)

    if day_plan.total_cost > daily_budget:
        scale = daily_budget / day_plan.total_cost
        for activity in day_plan.activities:
            activity.estimated_cost *= scale
        day_plan.total_cost = daily_budget

    return day_plan

def detect_conflicts(destination: str, num_days: int, total_budget: float,
                     daily_plans: List[DayPlan]) -> ConflictResolution:
    """Detect conflicts including duplicate locations"""
    total_spent = sum(day.total_cost for day in daily_plans)

    # Track locations per day for duplicate detection
    location_days = {}
    for day in daily_plans:
        for activity in day.activities:
            loc = activity.location.lower().strip()
            if loc not in location_days:
                location_days[loc] = []
            location_days[loc].append(day.day_number)

    # Find duplicates
    duplicate_locations = {loc: days for loc, days in location_days.items() if len(days) > 1}

    itinerary_summary = "\n\n".join([
        f"Day {day.day_number} (${day.total_cost:.2f}):\n" +
        "\n".join([f"  {a.time_of_day}: {a.name} at {a.location} ({a.duration_hours}h)"
                   for a in day.activities])
        for day in daily_plans
    ])

    duplicate_info = ""
    if duplicate_locations:
        duplicate_info = "\n\nDUPLICATE LOCATIONS FOUND:\n" + "\n".join(
            [f"- '{loc}' visited on days: {days}" for loc, days in duplicate_locations.items()]
        )

    review_prompt = f"""Analyze this {num_days}-day {destination} itinerary:

{itinerary_summary}
{duplicate_info}

Budget: ${total_budget:.2f} | Spent: ${total_spent:.2f}

Identify conflicts:
1. DUPLICATE LOCATIONS - Same location visited multiple times (CRITICAL)
2. Time conflicts (overlapping activities, insufficient travel time)
3. Budget issues (over budget, unrealistic costs)
4. Logical issues (missing meals, poor pacing, activities at wrong time)
5. Location issues (too much travel, illogical routing)

For each issue, specify:
- What's wrong (include specific location names if duplicates)
- How to fix it (suggest alternative locations)
"""

    structured_llm = llm.with_structured_output(ConflictResolution)
    return structured_llm.invoke([
        SystemMessage(content="You are a travel planner identifying itinerary conflicts, especially duplicate locations."),
        HumanMessage(content=review_prompt)
    ])

def resolve_conflicts(destination: str, num_days: int, total_budget: float,
                      daily_plans: List[DayPlan], conflicts: ConflictResolution) -> List[DayPlan]:
    """Automatically resolve conflicts ensuring unique locations"""
    if not conflicts.has_conflicts:
        return daily_plans

    fixed_plans = []
    remaining_budget = total_budget
    target_utilization = 0.92

    for day_num in range(1, num_days + 1):
        remaining_days = num_days - day_num + 1
        daily_limit = remaining_budget / remaining_days
        target_cost = daily_limit * target_utilization

        # Get locations already used in fixed days
        used_locations = get_visited_locations(fixed_plans)
        used_list = "\n".join([f"- {loc}" for loc in used_locations]) if used_locations else "None yet"

        original_day = daily_plans[day_num - 1]

        fix_prompt = f"""Fix Day {day_num} of a {num_days}-day trip to {destination}.

LOCATIONS ALREADY USED IN OTHER DAYS (AVOID THESE):
{used_list}

ISSUES TO RESOLVE:
{chr(10).join(f'- {issue}' for issue in conflicts.issues_found if f'Day {day_num}' in issue or 'duplicate' in issue.lower())}

RESOLUTIONS NEEDED:
{chr(10).join(f'- {res}' for res in conflicts.resolutions)}

ORIGINAL DAY {day_num} ACTIVITIES:
{chr(10).join(f'{a.time_of_day}: {a.name} at {a.location} (${a.estimated_cost:.2f})' for a in original_day.activities)}

Daily Budget: ${daily_limit:.2f}
TARGET: ${target_cost:.2f} (90-95% of budget)

CRITICAL REQUIREMENTS:
1. Use COMPLETELY DIFFERENT locations not in the "already used" list
2. Choose NEW restaurants, NEW attractions, NEW areas
3. Each location must be UNIQUE across all days
4. Resolve all timing and logical conflicts
5. Aim for ${target_cost:.2f} total cost
"""

        structured_llm = llm.with_structured_output(DayPlan)
        fixed_day = structured_llm.invoke([
            SystemMessage(content="You fix travel itinerary conflicts, especially duplicate locations."),
            HumanMessage(content=fix_prompt)
        ])

        # Budget adjustment
        if fixed_day.total_cost < daily_limit * 0.85:
            scale = target_cost / fixed_day.total_cost if fixed_day.total_cost > 0 else 1
            for activity in fixed_day.activities:
                activity.estimated_cost *= scale
            fixed_day.total_cost = min(target_cost, daily_limit)

        if fixed_day.total_cost > daily_limit:
            scale = daily_limit / fixed_day.total_cost
            for activity in fixed_day.activities:
                activity.estimated_cost *= scale
            fixed_day.total_cost = daily_limit

        fixed_plans.append(fixed_day)
        remaining_budget -= fixed_day.total_cost

    return fixed_plans

print("Helper functions defined!")

Helper functions defined!


## Main Itinerary Generator

In [7]:
def create_itinerary(destination: str, total_budget: float):
    """Create travel itinerary with unique locations per day"""

    print(f"\n{'='*60}")
    print(f"Creating itinerary for {destination}")
    print(f"Total Budget: ${total_budget:.2f}")
    print(f"{'='*60}\n")

    num_days = 3
    budget_per_day = total_budget / num_days
    print(f"Trip duration: {num_days} days (${budget_per_day:.2f}/day)\n")

    print("Researching destination...")
    guide = query_destination_guide(destination, budget_per_day)
    print("Research complete!\n")

    # Plan each day with location tracking
    print("Planning itinerary with unique locations...")
    daily_plans = []
    remaining_budget = total_budget

    for day in range(1, num_days + 1):
        remaining_days = num_days - day + 1
        daily_limit = remaining_budget / remaining_days

        # Get visited locations from previous days
        visited = get_visited_locations(daily_plans)

        day_plan = plan_single_day(
            destination, day, num_days, daily_limit, guide, visited
        )
        daily_plans.append(day_plan)
        remaining_budget -= day_plan.total_cost
        print(f"   Day {day}: ${day_plan.total_cost:.2f} ({len(day_plan.activities)} activities)")

    print("Initial plan complete!\n")

    # Detect and resolve conflicts
    print("Analyzing for conflicts...")
    conflicts = detect_conflicts(destination, num_days, total_budget, daily_plans)

    if conflicts.has_conflicts:
        print(f"Found {len(conflicts.issues_found)} issues")
        print("Resolving conflicts...")
        daily_plans = resolve_conflicts(destination, num_days, total_budget, daily_plans, conflicts)
        print("All conflicts resolved!\n")
    else:
        print("No conflicts found!\n")

    # Calculate totals
    total_spent = sum(day.total_cost for day in daily_plans)
    utilization = (total_spent / total_budget) * 100

    print(f"{'='*60}")
    print(f"Total Spent: ${total_spent:.2f} ({utilization:.1f}% of budget)")
    print(f"Remaining: ${total_budget - total_spent:.2f}")
    print(f"{'='*60}\n")

    # Build itinerary
    itinerary = f"""# {destination.title()} Travel Itinerary

## Trip Overview

**Duration:** {num_days} Days
**Total Budget:** ${total_budget:.2f}
**Estimated Cost:** ${total_spent:.2f}
**Budget Utilization:** {utilization:.1f}%
**Remaining:** ${total_budget - total_spent:.2f}

---

## Daily Itinerary

"""

    for day in daily_plans:
        itinerary += f"""### Day {day.day_number}

**Daily Budget:** ${day.total_cost:.2f}

"""
        for activity in day.activities:
            itinerary += f"""#### {activity.time_of_day} - {activity.name}

- **Location:** {activity.location}
- **Duration:** {activity.duration_hours} hours
- **Cost:** ${activity.estimated_cost:.2f}
- **Category:** {activity.category}

{activity.description}

"""
        itinerary += "---\n\n"

    return itinerary, daily_plans, conflicts

print("Main function ready!")

Main function ready!


## Create Your Itinerary

In [8]:
# Get user input
city = input("Enter your destination: ")
budget = float(input("Enter your budget (USD): "))

# Generate optimized itinerary
itinerary, daily_plans, conflicts = create_itinerary(city, budget)

# Display the final itinerary
display(Markdown(itinerary))

# Save to file
with open('final_itinerary.md', 'w', encoding='utf-8') as f:
    f.write(itinerary)

print("\nFinal optimized itinerary saved to 'final_itinerary.md'!")


Creating itinerary for Shimla
Total Budget: $80.00

Trip duration: 3 days ($26.67/day)

Researching destination...
Research complete!

Planning itinerary with unique locations...
   Day 1: $24.53 (6 activities)
   Day 2: $25.51 (6 activities)
   Day 3: $26.50 (6 activities)
Initial plan complete!

Analyzing for conflicts...
Found 2 issues
Resolving conflicts...
All conflicts resolved!

Total Spent: $77.02 (96.3% of budget)
Remaining: $2.98



# Shimla Travel Itinerary

## Trip Overview

**Duration:** 3 Days
**Total Budget:** $80.00
**Estimated Cost:** $77.02
**Budget Utilization:** 96.3%
**Remaining:** $2.98

---

## Daily Itinerary

### Day 1

**Daily Budget:** $24.49

#### Morning - Breakfast and Stroll

- **Location:** Indian Coffee House, Mall Road, Shimla
- **Duration:** 2.0 hours
- **Cost:** $2.58
- **Category:** Food

Enjoy a hearty breakfast followed by a leisurely stroll along the scenic Mall Road.

#### Morning - Visit Jakhoo Temple

- **Location:** Jakhoo Temple, Jakhoo Hill, Shimla
- **Duration:** 2.0 hours
- **Cost:** $0.00
- **Category:** Sightseeing

Explore the historic Jakhoo Temple, known for its panoramic views and large Hanuman statue.

#### Afternoon - Lunch at Cafe Simla Times

- **Location:** Cafe Simla Times, The Mall, Shimla
- **Duration:** 1.5 hours
- **Cost:** $4.00
- **Category:** Food

Relish a delicious lunch at Cafe Simla Times, known for its vibrant ambiance and diverse menu.

#### Afternoon - Explore Lakkar Bazaar

- **Location:** Lakkar Bazaar, Shimla
- **Duration:** 1.5 hours
- **Cost:** $0.00
- **Category:** Sightseeing

Wander through Lakkar Bazaar, famous for its wooden crafts and souvenirs.

#### Evening - Dinner at Wake & Bake Cafe

- **Location:** Wake & Bake Cafe, The Mall, Shimla
- **Duration:** 1.5 hours
- **Cost:** $5.00
- **Category:** Food

Enjoy a cozy dinner at Wake & Bake Cafe, offering a variety of international cuisines.

#### Evening - Stay at Budget Homestay

- **Location:** Budget Homestay, Chotta Shimla, Shimla
- **Duration:** 8.0 hours
- **Cost:** $12.91
- **Category:** Accommodation

Relax and unwind at a comfortable budget homestay in Chotta Shimla.

---

### Day 2

**Daily Budget:** $25.53

#### Morning - Breakfast at Hide Out Cafe

- **Location:** Hide Out Cafe, Middle Bazar, Shimla
- **Duration:** 1.5 hours
- **Cost:** $5.46
- **Category:** Food

Start your day with a hearty breakfast at this cozy cafe known for its delicious pancakes and coffee.

#### Morning - Visit to Viceregal Lodge

- **Location:** Viceregal Lodge, Observatory Hill, Shimla
- **Duration:** 2.0 hours
- **Cost:** $2.61
- **Category:** Sightseeing

Explore the historic Viceregal Lodge, a beautiful example of British architecture and a significant landmark in Shimla.

#### Afternoon - Lunch at Ashiana & Goofa Restaurant

- **Location:** Ashiana & Goofa Restaurant, The Ridge, Shimla
- **Duration:** 1.5 hours
- **Cost:** $7.64
- **Category:** Food

Enjoy a traditional Himachali meal with a view of the city at this popular restaurant.

#### Afternoon - Trek to Chadwick Falls

- **Location:** Chadwick Falls, Shimla
- **Duration:** 3.0 hours
- **Cost:** $0.00
- **Category:** Activity

Take a refreshing trek to the scenic Chadwick Falls, a serene spot surrounded by lush greenery.

#### Evening - Dinner at Eighteen71 Cookhouse & Bar

- **Location:** Eighteen71 Cookhouse & Bar, Hotel Willow Banks, Shimla
- **Duration:** 2.0 hours
- **Cost:** $9.83
- **Category:** Food

Dine at this modern restaurant offering a variety of cuisines and a vibrant atmosphere.

#### Evening - Evening Walk at Annandale

- **Location:** Annandale, Shimla
- **Duration:** 1.0 hours
- **Cost:** $0.00
- **Category:** Activity

Enjoy a peaceful evening walk at Annandale, a picturesque flat terrain surrounded by deodar trees.

---

### Day 3

**Daily Budget:** $27.00

#### Morning - Breakfast at Cafe Sol

- **Location:** Cafe Sol, The Mall, Shimla
- **Duration:** 1.5 hours
- **Cost:** $4.50
- **Category:** Food

Enjoy a hearty breakfast with a view of the hills.

#### Morning - Visit to Himalayan Bird Park

- **Location:** Himalayan Bird Park, Shimla
- **Duration:** 2.0 hours
- **Cost:** $2.50
- **Category:** Sightseeing

Explore the diverse bird species in a serene environment.

#### Afternoon - Lunch at Baljees

- **Location:** Baljees, The Mall, Shimla
- **Duration:** 1.5 hours
- **Cost:** $6.00
- **Category:** Food

Savor traditional Indian cuisine at a popular local restaurant.

#### Afternoon - Trek to Glen

- **Location:** Glen, Shimla
- **Duration:** 2.5 hours
- **Cost:** $0.00
- **Category:** Activity

Enjoy a scenic trek through the lush green forest of Glen.

#### Evening - Dinner at 45 Central Perk

- **Location:** 45 Central Perk, Shimla
- **Duration:** 2.0 hours
- **Cost:** $14.00
- **Category:** Food

Dine at a cozy cafe with a variety of cuisines.

---




Final optimized itinerary saved to 'final_itinerary.md'!


## View Conflict Resolution Details

In [9]:
if 'conflicts' in locals():
    print("\nConflict Resolution Report:\n")

    if conflicts.has_conflicts:
        print(f"Issues Found ({len(conflicts.issues_found)}):")
        for i, issue in enumerate(conflicts.issues_found, 1):
            print(f"  {i}. {issue}")

        print(f"\nResolutions Applied ({len(conflicts.resolutions)}):")
        for i, resolution in enumerate(conflicts.resolutions, 1):
            print(f"  {i}. {resolution}")
    else:
        print("No conflicts were found - itinerary was already optimized!")
else:
    print("No conflict data available. Run the itinerary generator first!")


Conflict Resolution Report:

Issues Found (2):
  1. Duplicate location: 'The Ridge, Shimla' visited multiple times on Day 3.
  2. Duplicate location: 'Mall Road, Shimla' visited multiple times on Day 1 and Day 2.

Resolutions Applied (3):
  1. Consider combining the visit to Christ Church and breakfast at Honey Hut into a single activity on Day 3 to reduce redundancy at 'The Ridge, Shimla'.
  2. On Day 1, combine the walk on Mall Road with breakfast at Indian Coffee House to streamline the itinerary.
  3. On Day 2, consider replacing the evening stroll at Scandal Point with a different activity to avoid redundancy on 'The Mall, Shimla'.


## Cost Breakdown by Category

In [10]:
if 'daily_plans' in locals():
    print("\nCost Breakdown:\n")

    categories = {}
    for day in daily_plans:
        for activity in day.activities:
            cat = activity.category
            if cat not in categories:
                categories[cat] = 0
            categories[cat] += activity.estimated_cost

    total = sum(categories.values())
    for category, cost in sorted(categories.items(), key=lambda x: x[1], reverse=True):
        percentage = (cost / total) * 100
        print(f"  {category:20s}: ${cost:7.2f} ({percentage:5.1f}%)")

    print(f"\n  {'TOTAL':20s}: ${total:7.2f}")
else:
    print("No itinerary data. Run the generator first!")


Cost Breakdown:

  Food                : $  59.01 ( 76.6%)
  Accommodation       : $  12.91 ( 16.8%)
  Sightseeing         : $   5.11 (  6.6%)
  Activity            : $   0.00 (  0.0%)

  TOTAL               : $  77.02
